<a href="https://colab.research.google.com/github/raulinar/information_retrieval_/blob/main/Project_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df_news = pd.read_csv('FinalData.csv')
df_news.head()

,judul,teks
0,Jaksa Minta Hakim Tolak Eksepsi Pinangki,"Jakarta, CNN Indonesia --Jaksa Penuntut Umum p..."
1,Alasan DPRD DKI Rapat di Puncak: Demi Cegah Covid,"Jakarta, CNN Indonesia --Pelaksana Tugas Sekre..."
2,KPAI: 171 Pelajar Ditangkap Ikut Demo 20 Oktob...,"Jakarta, CNN Indonesia --Komisi Perlindungan A..."
3,Jepang Minta Solusi Damai Sengketa Laut China ...,"Jakarta, CNN Indonesia --Jepang meminta kepada..."
4,AS Sebut Butuh Indonesia untuk Lawan Dominasi ...,"Jakarta, CNN Indonesia --Menteri Pertahanan Am..."


In [ ]:
df_news = df_news.drop('judul', axis=1)
df_news.head()

,teks
0,"Jakarta, CNN Indonesia --Jaksa Penuntut Umum p..."
1,"Jakarta, CNN Indonesia --Pelaksana Tugas Sekre..."
2,"Jakarta, CNN Indonesia --Komisi Perlindungan A..."
3,"Jakarta, CNN Indonesia --Jepang meminta kepada..."
4,"Jakarta, CNN Indonesia --Menteri Pertahanan Am..."


In [ ]:
df_news['id'] = range(1, len(df_news)+1)
df_news.head()

,teks,id
0,"Jakarta, CNN Indonesia --Jaksa Penuntut Umum p...",1
1,"Jakarta, CNN Indonesia --Pelaksana Tugas Sekre...",2
2,"Jakarta, CNN Indonesia --Komisi Perlindungan A...",3
3,"Jakarta, CNN Indonesia --Jepang meminta kepada...",4
4,"Jakarta, CNN Indonesia --Menteri Pertahanan Am...",5


In [ ]:
# df_news = df_news[:209]
# df_news.head()

In [ ]:
#@title STOPWORD

In [ ]:
f = open("stopword_list_tala.txt", "r")
isi = f.read()

tempStoplist = []
for tempstp in isi.split():
  tempStoplist.append(tempstp)

In [ ]:
tempStoplist

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
#@title Stemming

In [ ]:
pip install sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
#@title Pengaplikasian Stopword, Stemming


In [ ]:
def preprocess(teks):
  cleantext = "@\S+|https?:\S+|http?:\S|[^A-Za-z]+"
  teks = re.sub(cleantext,' ',str(teks).lower()).strip()
  tokens = []
  for token in teks.split():
    if token not in tempStoplist:
      token = stemmer.stem(token)
      tokens.append(token)
  return " ".join(tokens)

In [ ]:
df_news['teks'] = df_news.teks.apply(lambda x:preprocess(x))
df_news.head()

,teks,id
0,jakarta cnn indonesia jaksa tuntut jaksa agung...,1
1,jakarta cnn indonesia laksana tugas sekretaris...,2
2,jakarta cnn indonesia komisi lindung anak indo...,3
3,jakarta cnn indonesia jepang libat sengketa ai...,4
4,jakarta cnn indonesia menteri tahan amerika se...,5


In [ ]:
#@title Inverted Index


In [ ]:
# class Appearance

class Appearance:
    def __init__(self, docId, frequency):
        self.docId = docId
        self.frequency = frequency

    def __repr__(self):
        return str(self.__dict__)

In [ ]:
#  class InvertedIndex

def invertedIndex(index, document):
        artikel = document['teks']
        terms = artikel.split(' ')
        appearances_dict = dict()

        for term in terms:
          term_frequency = appearances_dict[term].frequency if term in appearances_dict else 0
          appearances_dict[term] = Appearance(document['id'], term_frequency + 1)

        update_dict = { key: [appearance]
                      if key not in index 
                      else index[key] + [appearance]
                      for (key, appearance) in appearances_dict.items() }
        index.update(update_dict)

        return index

In [ ]:
indeks = {}
for row_dict in df_news.to_dict(orient="records"):
  invertedIndex(indeks, row_dict)

In [ ]:
indeks

{'jakarta': [{'docId': 1, 'frequency': 2},
  {'docId': 2, 'frequency': 8},
  {'docId': 3, 'frequency': 4},
  {'docId': 4, 'frequency': 2},
  {'docId': 5, 'frequency': 1},
  {'docId': 6, 'frequency': 1},
  {'docId': 7, 'frequency': 1},
  {'docId': 8, 'frequency': 1},
  {'docId': 9, 'frequency': 5},
  {'docId': 10, 'frequency': 1},
  {'docId': 11, 'frequency': 1},
  {'docId': 12, 'frequency': 1},
  {'docId': 13, 'frequency': 4},
  {'docId': 14, 'frequency': 1},
  {'docId': 15, 'frequency': 1},
  {'docId': 16, 'frequency': 3},
  {'docId': 17, 'frequency': 1},
  {'docId': 18, 'frequency': 1},
  {'docId': 19, 'frequency': 1},
  {'docId': 20, 'frequency': 1},
  {'docId': 21, 'frequency': 6},
  {'docId': 22, 'frequency': 2},
  {'docId': 23, 'frequency': 1},
  {'docId': 24, 'frequency': 1},
  {'docId': 25, 'frequency': 3},
  {'docId': 26, 'frequency': 3},
  {'docId': 29, 'frequency': 3},
  {'docId': 30, 'frequency': 1},
  {'docId': 31, 'frequency': 1},
  {'docId': 32, 'frequency': 1},
  {'docI

In [ ]:
#@title Menghitung Korpus


In [ ]:
N_dok = len(df_news.index)

In [ ]:
listTerm = []
bobot = []
idf = []

for term2 in indeks:
  listTerm.append(term2)
  
  tfidf = [0]*N_dok

  df = len(indeks[term2])
  idf.append(np.log(N_dok/df))
  for v in indeks[term2]:
    tfidf[v.docId - 1] = v.frequency*(idf[listTerm.index(term2)])
  bobot.append(tfidf) 

In [ ]:
#@title Menghitung Bobot Kueri


In [ ]:
def preproccesingQuery(search_term):
  teksQ = search_term

  frekQ = [0]*len(indeks)
  for tokenQ in teksQ.split():
    if tokenQ in indeks.keys():
      i = listTerm.index(tokenQ)
      frekQ[i] = frekQ[i]+1

  tfidfQ = [0]*len(indeks)
  wq2 = 0
  for i in range(len(tfidfQ)):
    tfidfQ[i] = frekQ[i]*idf[i]
    wq2 = wq2 + np.power(tfidfQ[i], 2)

  panjangQ = np.sqrt(wq2)

  return panjangQ, tfidfQ

In [ ]:
#@title Scoring / Similarity


In [ ]:
def similarity(panjangQ, tfidfQ):
  score =[0]*(N_dok)

  print("Similarity kueri dengan setiap dokumen = ")
  for j in range(N_dok):
    wj2 = 0
    wjq = 0
    for i in range(len(bobot)):
      wj2 = wj2 + (np.power(bobot[i][j],2))
      wjq = wjq + (bobot[i][j]*tfidfQ[i])

    panjangW = np.sqrt(wj2)
    cossin = wjq/(panjangW*panjangQ)
    score[j] = 0 if np.isnan(cossin) else cossin

    print("Dok ", (j+1), ": ", score[j])
  return score

In [ ]:
search_term = input("Masukan Kueri : ")

hasilPreproKueri = preproccesingQuery(search_term)
score = similarity(hasilPreproKueri[0], hasilPreproKueri[1])

In [ ]:
#@title Mengurutkan Dokumen Sesuai Nilai Similarity


In [ ]:
urut = sorted(range(len(score)), key=lambda k: score[k]+1, reverse=True)
for i in urut:
  print("dokumen ",i+1,": ", df_news["teks"][df_news['id'] == i+1]. values[0])